In [1]:
import numpy as np
import pandas as pd
import time
from model import Model

# ダウンロードしてきたやつ
INDEIES = ["N225",
           "AORD",
           "HSI",
           "GDAXI",
           "FTSE",
           "NYA",
           "DJI",
           "GSPC",
          ]
ANSWER = "N225"

def getClosing():
    closing = pd.DataFrame()
    for index in INDEIES:
        # na_valuesは文字列"null"のとき空として扱う CSVみるとnullって書いてあります。
        df = pd.read_csv("./data/input1/" + index + ".csv", na_values=["null"])
        df["Date"] = pd.to_datetime(df["Date"])
        df = df.set_index("Date")
        closing[index] = df["Close"]
    #空の部分は古いので埋める。
    closing = closing.fillna(method="ffill")
    for index in INDEIES:
        closing[index] = closing[index] / max(closing[index])
        closing[index] = np.log(closing[index] / closing[index].shift())
    closing["positive"] = 0
    #closing["N225"] >= 0の行のpositiveに1をいれる。
    closing.loc[closing[ANSWER] >= 0, "positive"] = 1
    closing["negative"] = 0
    #closing["N225"] < 0の行のnegativeに1をいれる。
    closing.loc[closing[ANSWER] < 0, "negative"] = 1
    return closing

def getTraningData():
    closing = getClosing()
    #1~3日前のデータを予測に使う
    days_before = range(1,4)
    answers = pd.DataFrame(columns = ["positive", "negative"])
    columns = []
    for index in INDEIES:
        columns += [index + "_" + str(before) for before in days_before]
    features = pd.DataFrame(columns = columns)
    #なんで7から？
    for i in range(7, len(closing)):
        #予測の部分は当日のデータで
        answers = answers.append({
            "positive" : closing["positive"].iloc[i],
            "negative" : closing["negative"].iloc[i]}, ignore_index=True)
        data={}
        #ほかの指標は１個前のデータを使用する。
        for index in INDEIES:
            for before in days_before :
                data[index + "_" + str(before)] = closing[index].iloc[i - before]
        features = features.append(data, ignore_index=True)
    #予測する元のデータ , 予測するべきデータ
    return features,answers

if __name__ == "__main__":
    start = time.time()
    features,answers = getTraningData()
    #[]は隠れ層なし、[50.25]は２層
    for layers in [[],[50,25]]:
        model = Model(features,answers,layers)
        model.train(30000)
        print('Accuracy = ',  model.test())
    print("elapsed_time :", time.time()-start, "[sec]")

3000 0.565061
6000 0.612193
9000 0.635246
12000 0.649078
15000 0.657787
18000 0.668033
21000 0.669057
24000 0.672643
27000 0.677766
30000 0.680328
Accuracy =  0.610655737705
3000 0.679816
6000 0.684939
9000 0.685451
12000 0.686988
15000 0.6875
18000 0.688525
21000 0.689037
24000 0.689549
27000 0.690061
30000 0.689037
Accuracy =  0.629098360656
elapsed_time : 190.89808797836304 [sec]
